In [1]:
# 0) Ir a la raíz del proyecto (ajusta la ruta si tu carpeta es distinta)
%cd /content/drive/MyDrive/Proyecto 4

# 1) Dependencias + instalación editable (para que 'src' sea importable)
%pip install -q -r requirements.txt
%pip install -q -e .

# 2) Parchear el dashboard para que siempre encuentre 'src' (solo si no lo tiene ya)
from pathlib import Path
app = Path("dashboard/streamlit_app.py")
code = app.read_text(encoding="utf-8")
bootstrap = """
# --- Path bootstrap BEFORE third-party imports ---
import sys
from pathlib import Path
ROOT = Path(__file__).resolve().parents[1]  # raíz del repo (contiene 'src/')
if (ROOT / "src").exists() and str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
"""
if "Path bootstrap BEFORE third-party imports" not in code:
    app.write_text(bootstrap + "\n" + code, encoding="utf-8")
    print("Dashboard parcheado para agregar ROOT en sys.path")
else:
    print("Dashboard ya tenía el bootstrap")

# 3) Asegurar que existe el dataset procesado; si no existe, lo generamos rápido
from pathlib import Path
Path("data/processed").mkdir(parents=True, exist_ok=True)
if not Path("data/processed/sales.csv").exists():
    print(" No se encontró data/processed/sales.csv — se generará uno sintético.")
    import pandas as pd, numpy as np
    from src.data.loaders import load_sales_data, load_external_data, assign_product_line
    from src.features.build_features import build_features

    sales = load_sales_data()
    ext = load_external_data()
    for d in (sales, ext):
        d["date"] = pd.to_datetime(d["date"], errors="coerce").dt.normalize()

    # Mantener SKU y línea de producto
    if "product_line" not in sales.columns or sales["product_line"].isna().all():
        sales["product_line"] = sales["sku"].map(assign_product_line) if "sku" in sales.columns else "Linea_UNICA"
        if "sku" not in sales.columns:
            sales["sku"] = "SKU_UNICO"

    num_cols = sales.select_dtypes(include=[np.number]).columns.tolist()
    agg = {c: ('sum' if c in ['sales'] else 'mean') for c in num_cols}
    sales_by = (sales.groupby(['date','sku','product_line'], as_index=False).agg(agg))
    df = sales_by.merge(ext, on='date', how='left')
    df = build_features(df)
    df.to_csv("data/processed/sales.csv", index=False)
    print("Guardado data/processed/sales.csv")

# 4) Lanzar Streamlit y obtener URL clicable en Colab
!streamlit run dashboard/streamlit_app.py --server.port 8501 --server.address 127.0.0.1 &> /content/streamlit.log &
from google.colab import output
print("Abre el dashboard aquí:", output.eval_js('google.colab.kernel.proxyPort(8501)'))


/content/drive/MyDrive/Proyecto 4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Dashboard ya tenía el bootstrap
Abre el dashboard aquí: https://8501-m-s-1j66xi4b4vrwq-b.europe-west4-0.prod.colab.dev


In [3]:
# 1) Ir a la raíz del proyecto
%cd /content/drive/MyDrive/Proyecto 4

# 2) Asegurar instalación (por si reiniciaste runtime)
%pip install -q -r requirements.txt
%pip install -q -e .

# 3) Matar procesos viejos de streamlit (si los hay)
!pkill -f "streamlit run" || true

# 4) Lanzar con flags que evitan bloqueos de CORS/proxy en Colab
!streamlit run dashboard/streamlit_app.py \
  --server.port 8501 \
  --server.address 0.0.0.0 \
  --server.enableCORS false \
  --server.enableXsrfProtection false \
  --server.headless true \
  &> /content/streamlit.log &

# 5) Obtener la URL proxificada de Colab
from google.colab import output
print(" Abre aquí:", output.eval_js('google.colab.kernel.proxyPort(8501)'))

# 6) Ver últimos logs (útil si queda colgado)
import time, subprocess, textwrap
time.sleep(2)
print("—— Log —————————————————————————————————————————")
print(subprocess.run(["tail","-n","50","/content/streamlit.log"], capture_output=True, text=True).stdout)


/content/drive/MyDrive/Proyecto 4
  Preparing metadata (setup.py) ... done
^C
 Abre aquí: https://8501-m-s-1j66xi4b4vrwq-b.europe-west4-0.prod.colab.dev
—— Log —————————————————————————————————————————



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501


